In [1]:
!pip install nltk sentence_transformers language_tool_python transformers torch torchvision torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag
from nltk.corpus import wordnet as wn
from sentence_transformers import SentenceTransformer, util
import random
import language_tool_python
from transformers import MarianMTModel, MarianTokenizer, T5ForConditionalGeneration, T5Tokenizer
import torch


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
import nltk

In [36]:
 nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [35]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

[('I', 'PRP'), ('am', 'VBP'), ('a', 'DT'), ('student', 'NN')]

design 

1. The basics functions , word tokens and pos_tags, finding synonyms, checking similarity of words an their function class DatareProcessing 
2. The paraphasing class, for back_tracking and building the paraphased text with similar structure and grammer check
3. Text processing 





In [37]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag
from nltk.corpus import wordnet as wn
from sentence_transformers import SentenceTransformer, util
import random
import language_tool_python
from transformers import MarianMTModel, MarianTokenizer, T5ForConditionalGeneration, T5Tokenizer
import torch

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [41]:

class DataPreprocessing:
    def __init__(self):
        self.model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

    def tag(self, sentence): # assigns pos_tags to the tokenized words 
        words = word_tokenize(sentence)
        words = pos_tag(words)
        return words

    def pos(self, tag):# converts nltk pos_tags to wordnet pos_tags to be used further
        if tag.startswith('NN'):
            return wn.NOUN
        elif tag.startswith('VB'):
            return wn.VERB
        elif tag.startswith('JJ'):
            return wn.ADJ
        elif tag.startswith('RB'):
            return wn.ADV
        else:
            return None

    def synonyms(self, word, tag):# find synonymns for the word and the tag
        try:
            pos = self.pos(tag)
            if pos:
                synsets = wn.synsets(word, pos=pos)
                lemmas = {lemma.name().replace('_',' ') for synset in synsets for lemma in synset.lemmas()}
                return lemmas
            else:
                return set()
        except Exception as e:
            print(f"Error finding synonyms for {word}: {e}")
            return set()

    def controlled_set_similarity(self, word, similar_words): 
        # converts words and its synonyms into embeddings using paraphrase-MiniLM-L6-v2 model and only keeps the ones with similarity score > 0.75
        try: 
            word_embedding = self.model.encode(word, convert_to_tensor=True)
            similar_words = {sw for sw in similar_words if util.pytorch_cos_sim(word_embedding, self.model.encode(sw, convert_to_tensor=True)).item() >= 0.75}
            return similar_words
        except Exception as e:
            print(f"Error in similarity control set for {word}: {e}")
            return similar_words

    def paraphraseable(self, tag):# which tags to generate synonyms for 
        return tag.startswith('NN') or tag.startswith('VB') or tag.startswith('JJ')

class Paraphrasing:
    # to generate the paraphased text , it makes the paraphased sentence utilising the preprocessor functions and then back_translate the paraphased sentence
    # and checks if it's grammatically correct 
    def __init__(self, preprocessor):
        # initialises all the tokenizers and models for back_tracking , and takes in a preporcessor object 
        self.preprocessor = preprocessor
        self.language_tool = language_tool_python.LanguageTool('en-US')
        self.back_translation_model_name = 'Helsinki-NLP/opus-mt-en-de'
        self.back_translation_model = MarianMTModel.from_pretrained(self.back_translation_model_name)
        self.back_translation_tokenizer = MarianTokenizer.from_pretrained(self.back_translation_model_name)
        self.back_translation_reverse_model_name = 'Helsinki-NLP/opus-mt-de-en'
        self.back_translation_reverse_model = MarianMTModel.from_pretrained(self.back_translation_reverse_model_name)
        self.back_translation_reverse_tokenizer = MarianTokenizer.from_pretrained(self.back_translation_reverse_model_name)

    def back_translate(self, sentence):
        # takes in a paraphased sentence as an input translates the test into german and again back to english 
        inputs = self.back_translation_tokenizer.encode(sentence, return_tensors="pt", max_length=512, truncation=True)
        outputs = self.back_translation_model.generate(inputs, max_length=512, num_return_sequences=1, num_beams=5, early_stopping=True)
        translated_sentence = self.back_translation_tokenizer.decode(outputs[0], skip_special_tokens=True)

        inputs = self.back_translation_reverse_tokenizer.encode(translated_sentence, return_tensors="pt", max_length=512, truncation=True)
        outputs = self.back_translation_reverse_model.generate(inputs, max_length=512, num_return_sequences=1, num_beams=5, early_stopping=True)
        back_translated_sentence = self.back_translation_reverse_tokenizer.decode(outputs[0], skip_special_tokens=True)

        return back_translated_sentence

    def synonym_if_exists(self, sentence):
        tagged_sentence = self.preprocessor.tag(sentence)
        paraphrased_sentence = []

        for word, pos_tag in tagged_sentence:
            if self.preprocessor.paraphraseable(pos_tag) and word.lower() not in ["i"]:
                word_synonyms = self.preprocessor.synonyms(word, pos_tag)
                if word_synonyms:
                    filtered_synonyms = self.preprocessor.controlled_set_similarity(word, word_synonyms)
                    if filtered_synonyms:
                        selected_synonym = random.choice(list(filtered_synonyms))
                        paraphrased_sentence.append(selected_synonym)
                    else:
                        paraphrased_sentence.append(word)# if not filtered_sunonyms
                else:
                    paraphrased_sentence.append(word)# if no synonymns
            else:
                paraphrased_sentence.append(word)# append words like we , are ,  i.e not paraphrasable words

        paraphrased_sentence = ' '.join(paraphrased_sentence)
        return paraphrased_sentence

    def correct(self, sentence):
        # corrects the sentence grammatically 
        return self.language_tool.correct(sentence)

class TextParaphraser:
    # this is the class that return the paraphased text and checks for the constraints on the input and output texts 
    def __init__(self):
        self.preprocessor = DataPreprocessing()
        self.paraphraser = Paraphrasing(self.preprocessor)

    def paraphrase_section(self, section):
        sentences = sent_tokenize(section) # 1. this converts paras into sentences
        paraphrased_sentences = []

        for sentence in sentences:
            paraphrased_sentence = self.paraphraser.synonym_if_exists(sentence)# 2. self.paraphraser creates a class object from Paraphrasing and initialises it
            # with a preprocess from the class DataPreprocessing
            
            paraphrased_sentence = self.paraphraser.back_translate(paraphrased_sentence)
            corrected_sentence = self.paraphraser.correct(paraphrased_sentence)
            paraphrased_sentences.append(corrected_sentence)

        return ' '.join(paraphrased_sentences)

    def paraphrase_text(self, text):
        sections = text.split('\n')
        paraphrased_text = []
        # keeping the sutructure of the document in contxt , setting up sections is necessary 
        word_count = len(text.split())
        if 200 <= word_count <= 400:
            for section in sections:
                section = section.strip()
                if not section:
                    continue
                
                paraphrased_section = self.paraphrase_section(section)
                paraphrased_text.append(paraphrased_section)
            
            paraphrased_text = '\n'.join(paraphrased_text)
            paraphrased_word_count = len(paraphrased_text.split())

            if paraphrased_word_count < 0.8 * word_count:
                print(f"Paraphrased text is too short: {paraphrased_word_count} words. Minimum required: {int(0.8 * word_count)} words.")
                return text  # returns the original text if paraphrased text is too short
            else:
                return paraphrased_text
        else:
            print(f"Text length out of bounds: {word_count} words. Skipping paraphrasing.")
            return text  # return the original text if it doesn't meet the length requirements

class ParaphrasingWithLLM:
    def __init__(self):
        self.tokenizer = T5Tokenizer.from_pretrained('t5-base')
        self.model = T5ForConditionalGeneration.from_pretrained('t5-base')

    def paraphrase(self, sentence):
        inputs = self.tokenizer.encode(f"paraphrase: {sentence}", return_tensors="pt", max_length=512, truncation=True)
        outputs = self.model.generate(inputs, max_length=512, num_return_sequences=1, num_beams=5, early_stopping=True)
        paraphrased_sentence = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return paraphrased_sentence




In [42]:
original_text = """A cover letter is a formal document that accompanies your resume when you apply for a job. It serves as an introduction and provides additional context for your application. Here’s a breakdown of its various aspects:\n
Purpose\n
The primary purpose of a cover letter is to introduce yourself to the hiring manager and to provide context for your resume. It allows you to elaborate on your qualifications, skills, and experiences in a way that your resume may not fully capture. It’s also an opportunity to express your enthusiasm for the role and the company, and to explain why you would be a good fit.\n
Content\n
A typical cover letter includes the following sections:\n
1. Header: Includes your contact information, the date, and the employer’s contact information.\n
2. Salutation: A greeting to the hiring manager, preferably personalized with their name.\n
3. Introduction: Briefly introduces who you are and the position you’re applying for.\n
4. Body: This is the core of your cover letter where you discuss your qualifications, experiences, and skills that make you suitable for the job. You can also mention how you can contribute to the company.\n
5. Conclusion: Summarizes your points and reiterates your enthusiasm for the role. You can also include a call to action, like asking for an interview.\n
6. Signature: A polite closing (“Sincerely,” “Best regards,” etc.) followed by your name.\n
Significance in the Job Application Process\n
The cover letter is often the first document that a hiring manager will read, so it sets the tone for your entire application. It provides you with a chance to stand out among other applicants and to make a strong first impression. Some employers specifically require a cover letter, and failing to include one could result in your application being disregarded.\n
In summary, a cover letter is an essential component of a job application that serves to introduce you, elaborate on your qualifications, and make a compelling case for why you should be considered for the
position."""
#print(original_text)


In [ ]:
text1 = """In the world of technology, artificial intelligence (AI) is making rapid advancements, transforming industries and reshaping the future. AI systems are capable of learning, reasoning, and self-correcting, which allows them to perform tasks that were previously thought to be exclusive to humans. One of the most significant applications of AI is in the field of healthcare. AI-powered systems can analyze vast amounts of medical data to identify patterns and predict outcomes, which can lead to more accurate diagnoses and personalized treatment plans. For instance, AI algorithms can examine medical images to detect anomalies that might be missed by the human eye, or analyze genetic information to predict the likelihood of certain diseases.

Another area where AI is having a substantial impact is in transportation. Self-driving cars, powered by AI, are becoming increasingly common and are expected to revolutionize the way we travel. These vehicles use sensors and complex algorithms to navigate roads, avoid obstacles, and make real-time decisions, all without human intervention. This technology has the potential to significantly reduce traffic accidents and improve road safety.

AI is also playing a crucial role in environmental conservation. Machine learning models can analyze satellite imagery to monitor deforestation, track wildlife populations, and predict natural disasters. This information is invaluable for conservation efforts and disaster preparedness, enabling quicker and more effective responses to environmental challenges.

Despite these advancements, the integration of AI into society comes with ethical and practical challenges. Concerns about privacy, security, and job displacement are prominent. As AI systems become more autonomous, ensuring they operate transparently and within ethical guidelines is paramount. Moreover, there is a need for continuous monitoring and regulation to prevent misuse and to protect public interest.

In summary, AI is a transformative technology with the potential to revolutionize various sectors, from healthcare and transportation to environmental conservation. However, it is essential to address the associated ethical and practical challenges to fully harness its benefits for society.

"""

In [ ]:
text2 = """Education is a vital part of a child's development, and schools play a crucial role in this process. Schools are not just places where children go to learn academic subjects; they are also environments where students develop social skills, learn to work collaboratively, and build their character. The structure and organization of a school can significantly impact the effectiveness of education.

Classrooms are the heart of the school where most of the learning takes place. Teachers guide students through the curriculum, providing instruction and support. In addition to academic lessons, teachers often incorporate activities that promote critical thinking, creativity, and problem-solving skills. These activities can range from group projects and discussions to hands-on experiments and artistic endeavors.

Extracurricular activities are also an essential component of school life. Sports teams, music programs, and clubs offer students opportunities to pursue their interests and develop new talents. Participation in these activities can enhance students' self-esteem, teach them about teamwork, and provide a sense of belonging.

The school community extends beyond the students and teachers to include parents and guardians. Parental involvement is crucial for a student's success. When parents engage in school activities, attend meetings, and communicate with teachers, they reinforce the importance of education and show support for their children's academic efforts.

Furthermore, schools often serve as centers for the community, hosting events and providing resources for families. Community involvement in schools can create a supportive and nurturing environment for students, helping them to thrive both academically and personally.

In conclusion, schools are fundamental institutions that shape the future of children. They provide a structured environment for learning, foster personal growth, and build strong communities. The collective efforts of teachers, parents, and the community are essential to ensure that schools fulfill their mission of educating and developing well-rounded individuals
"""

In [ ]:
!pip install rouge

In [40]:
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
from sentence_transformers import SentenceTransformer, util
import language_tool_python
import time

In [ ]:
# the comparision 
custom_paraphraser = TextParaphraser()
custom_paraphrase = custom_paraphraser.paraphrase_text(original_text)

llm_paraphraser = ParaphrasingWithLLM()


def calculate_bleu(reference, candidate):
    reference = [reference.split()]
    candidate = candidate.split()
    return sentence_bleu(reference, candidate)

def calculate_rouge(reference, candidate):
    rouge = Rouge()
    scores = rouge.get_scores(candidate, reference)
    return scores[0]  

def calculate_semantic_similarity(model, sentence1, sentence2):
    embedding1 = model.encode(sentence1, convert_to_tensor=True)
    embedding2 = model.encode(sentence2, convert_to_tensor=True)
    return util.pytorch_cos_sim(embedding1, embedding2).item()

def count_grammar_errors(tool, sentence):
    matches = tool.check(sentence)
    return len(matches)

def measure_latency(generator, text):
    start_time = time.time()
    _ = generator(text)
    end_time = time.time()
    return end_time - start_time

sentence_transformer = SentenceTransformer('paraphrase-MiniLM-L6-v2')
language_tool = language_tool_python.LanguageTool('en-US')

llm_paraphrase = llm_paraphraser.paraphrase(original_text)

custom_bleu = calculate_bleu(original_text, custom_paraphrase)
custom_rouge = calculate_rouge(original_text, custom_paraphrase)
custom_semantic_similarity = calculate_semantic_similarity(sentence_transformer, original_text, custom_paraphrase)
custom_grammar_errors = count_grammar_errors(language_tool, custom_paraphrase)
custom_latency = measure_latency(custom_paraphraser.paraphrase_text, original_text)


llm_bleu = calculate_bleu(original_text, llm_paraphrase)
llm_rouge = calculate_rouge(original_text, llm_paraphrase)
llm_semantic_similarity = calculate_semantic_similarity(sentence_transformer, original_text, llm_paraphrase)
llm_grammar_errors = count_grammar_errors(language_tool, llm_paraphrase)
llm_latency = measure_latency(llm_paraphraser.paraphrase, original_text)

print("\nCustom Paraphrase Generator Metrics:")
print(f"BLEU Score: {custom_bleu}")
print(f"ROUGE Score: {custom_rouge}")
print(f"Semantic Similarity: {custom_semantic_similarity}")
print(f"Grammar Errors: {custom_grammar_errors}")
print(f"Latency: {custom_latency:.4f} seconds")

print("\nLLM Paraphrase Generator Metrics:")
print(f"BLEU Score: {llm_bleu}")
print(f"ROUGE Score: {llm_rouge}")
print(f"Semantic Similarity: {llm_semantic_similarity}")
print(f"Grammar Errors: {llm_grammar_errors}")
print(f"Latency: {llm_latency:.4f} seconds")

print(f"Custom Paraphrase: {custom_paraphrase}")
print(f"LLM Paraphrase: {llm_paraphrase}")

In [ ]:

def tag(sentence):
  words = word_tokenize(sentence)
  pos_tags = pos_tag(words)
  return pos_tags


def pos(tag):


['I am a student.',
 'maybe I work at even.',
 'ihbu9isve.',
 'dcdsijfbhE.',
 'FWFwvS']